<img src="https://lh3.googleusercontent.com/pw/AM-JKLVhTn_UySwMdfMwXvoq8l3VN7IkrY9cwtH2YJVMxAlMznUBWC9IpFtgPRIyfAXru4oykkYD-1WjWi0Ao5XgkB9JICvzDBcfn0L_5X2_KOOppsURK5DfSifCC-s7Vx5oQrBUn_BNWn_hfAPdhlVbKQGE=w1097-h235-no?authuser=0" alt="2021年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

# Pythonライブラリを使う - Biopython -


## はじめに

　Pythonの基礎では、変数や関数（print関数やlen関数、range関数）、構文（繰り返し構文、条件分岐構文）を勉強しました。それら基礎だけでは、ゲノム解析はおこなえません。ゲノム解析では、ほとんどの場合、ファイルを読み込み、中身のデータ（塩基配列データやSNP多型データなど）の処理をおこないます。

　今回、Pythonの基礎の練習として、ゲノム解析でよく取り扱うデータファイル（**FASTA**形式ファイルや**FASTQ**形式ファイル）をPythonで処理する方法を勉強していきます。


　


### 塩基配列データ形式 FASTAとFASTQ

　生物のゲノム配列はどのような形でファイルに記述されているか？ よく使われている記述形式は、**FASTA**と呼ばれる形式です。FASTA形式は、下記の例のように、塩基配列の名前と塩基配列本体がセットで記述されています。
- 例） 3つの塩基配列データ  
  - 配列名: `>`で始まる行
  - 配列本体: `>`より下の行。次の`>`までがひとつ分のデータ。 

```
>chromosome_01
CCGGAGCCGGAGCCAGCGCAGCCTGCCGCGGCAAGTGGCGCCTCCCAGGTCGACCTGGAC
GTGGTTCTCGGGGTGATGGGCGCGGGCGGCGCCGCCTCGGTCGGGTTCGAGGACGCGGCG
GCGCTGTTTGAGGAGGAGGAGGCGACCGTGGAGGAGGCAGCAGCGGCTTTCCGCGTCTTC
GACAGCAACGGCGACGGGTTCGTTGACGCCGGAGAGCTCCGGAGCGTGCTGAGATCGCTC
GGGTTCACCGCCGGCGTCGC
>chromosome_02
AAATTTTAAGTTTGAACATTTCAGGGAGGATTTACCTCGGATTATGAATCATAAAATCAA
GACAAATTTACGGTTGAGGTTGGAGTGCACAATTTATCGTAGACACCGCCCTGAAGTAGA
CCCAAACACGGGGAAGAAAATATGTGTGGTGGAAAAACTAAGCACACACCACATTTTGAA
CGAAGTGACCATCGATCGTGGTCCAAGTCCTTTTCTATCCATGTTAGAATTGTATGGTGA
CGGCTCATTAATGACCGTTGCGCAGGCGGACGGACTGATTGCTGCTACTCCGACTGGGTC
CACGGCCTATTCTTTGAGTGCAGGTGGGTCATTGGTATGCCCAACCGTCAATGCAATCGC
TTTAACACCCATTTGTCCACATGCATTGAGTTTCAGACCCATCATCTTACCAGAAAGTAT
AAATTTAAAAGTGAAAGTCTCGATGAAGTCAAGGGCTCCAGCATGGGCGGCTTTTGATGG
GAAAGATAGAATTGAATTGCAAAAAGGTGATTTTATAACCATATGCGCCAGCCCATATGC
TTTTCCAACCGTGGAAGCCTCGCCCGATGAGTTTATTAACAGTATCAGTCGACAACTAAA
CTGGAATGTGAGGGAACAACAAAAGTCCTTTACGCATATTTTGTCCCAAAAGAACCAAGA
A
>chromosome_03
GCATCACCTCTAGTAAAAAGAAAATCCTACGGCATGCCAATTTTAGGGAACTTTTAGTCT
TGGTCTCTGTTGTCGTCTTCATTATAGGCGAATTGATTATGGCCCATAGCTTTGAAATGT
ACGCGAGGTTCAGCCCGCTTGAATTTGGGCTGCATTAGCTTGTTAATTTCCAAAGGAAAA
AATATATTAACTCCTTGAGTATCGGTAGAGAGACTCGTAGGAAATCACTAATTGAAGAAG
TATCACTTCCACCTCCCCAGGTTGCCCCTGGAAGGTTTTACTTTTTCGTAAGTCTGTTT
```

　生物のゲノムDNAを次世代シーケンサーで読むと、多量の短い塩基配列（以下、**リード**）が得られます。リードデータは通常、**FASTQ**と呼ばれる形式で記述されています。なぜFASTA形式ではなく、FASTQ形式なのか？ 次世代シーケンサーから出力されるデータは、塩基配列の名前と塩基配列本体だけでなく、各塩基のクオリティ（正確性のスコア）の情報も出力され、FASTA形式ではそのクオリティ情報を記述できないからです。そこで、クオリティ情報を記述できるFASTQ形式がよく使われています。

　FASTQ形式は、4行でひとつの塩基配列データを記述しています。
- 例） 3つの塩基配列データ  
  - 配列名: `@`で始まる行
  - 配列本体: `@`の次の行
  - 各塩基のクオリティ: `+`の次の行（`+`は配列本体とクオリティ情報の間の区切り）
    - 「J」や「<」、「A」、「7」、「-」のそれぞれの文字には、Phredクオリティスコアというものが割り当てられており、各塩基の正確性を表しています。
    - [Phredクオリティスコアの対応表と正確性について](https://github.com/CropEvol/lecture/blob/master/textbook_2021/L03_phred_score.md)。

```
@READ_00001
TGCTGCGGAGCCCACGCCACGATCTTCCCGAACCCCTTCGTGCGGTCGAGGAATCCGTCGGGGAGGGATTTCACGGCGTAGTCTTTCAGCGACCAAAGAAAGGTGTGTCTGCAAATTTCCAATGCCTCGGCCAATACCGC
+
JJJJJJJJJJJ<JJJJJJJJJJJJJJJJJJJJJJJJJJJAJAFJ<JJJFJ-AJJAFJJFFFJJJJJ-AJF-FJJJJJJJJJJFFJJ7FJAJJJJFAJFJJ<-7-A--7F-<JJF<-<JJJAF-<--7)))AJJJJ-7-<F
@READ_00002
TTTCTCCCCATCAATCGACAATATTTCTTGCTTTCTCGCCTTCCCCCATCCAAAATCTGCACCGTAAAGATCTAGCTTGCTCGATCCTGCCACCGAAAAATATGATTTCTGCAACGCTTCACGAATCTCCAGCGACTATT
+
7JJJFFJJFJJJJJFJFJJFJ-JJJJFJJFFFJJJJJJJJJJJJJJJJJJFFJJJJJJJJFJJ<AFJJJAAJJ-AFJFFFFJJFJJJJJJJJJJJ7JAJ<FFJJJFFFFFJJAFF-AF-<-FAJ<F77-AFF<-)<)7--
@READ_00003
CAGCGCCTTCTCTTCTATAAGCACCCCTGCACCCCTACTCATTTTGTTGATAACATTGTCCCAATCCTCAAAAAATCTCTCTCCCAAACTCTACGCCACTTCCCCGCCCTCGCCGGAAACGTATGCTATCCCTTCGACTC
+
<FJJJ<JJ<JJ<JJJJJJJ<F-FJ-7FJ7-FJJJJFAFJFJJJJ-FJJJJ-AJJJJFJFJJJJFJJFJJF<FJFAJJFJFJFJAFFFFJFJJJJJJJ-F7--A7--7A7FJJJJ-A--FF--7F---AAJJF--7AFFJJ

```


　今回のテキストでは、これら形式のファイルを扱います。
- 例題: FASTQ形式ファイル
- 実習: FASTA形式ファイル


### 次世代シーケンス解析の流れ

　次世代シーケンサーから出力されたデータをどのように解析していくのか？　その次世代シーケンス解析の一般的な流れを少しだけ勉強しておきましょう。

　ここでは、以下の仮定で話をおこないます。
- ある生物種の個体Aのゲノム配列データ（FASTA形式ファイル）を持っている。
- 同種の生物の個体Bや個体C、個体D、・・・のそれぞれのゲノムDNAを得て、次世代シーケンシングをおこない、リードデータ（FASTQ形式ファイル）を得ている。

ゲノム解析の流れ: 
1. **リードのクオリティコントロール**:  
　個体Bのオリジナルリードデータには、シーケンシングアダプター配列を含んだリードや低クオリティのリードが少なからず含まれています。そのような配列を除去し、良いリードデータのみ残します。
1. **リードのアライメント**:  
　塩基の相同性をもとに、個体Bの各リードを個体Aのゲノム配列上に並べます。
1. **多型検出**:  
　ゲノム配列（個体A）とリードデータ（個体B）のあいだの塩基の違いを検出します。
1. 個体Cや個体Dなどのデータも同様に、クオリティコントロール、アライメント、多型検出をおこないます。
1. 全個体の多型情報をまとめたテーブルを作成します。
1. その多型情報テーブルを使って、研究内容に合わせた解析（遺伝子マッピングや多様性解析など）をおこないます。

<img src="https://lh3.googleusercontent.com/pw/AM-JKLViNK-9cLg8wOMokZH2-AWh2Mrlsm9BOT4oGU4C9QFFrU___O30S7jiUUavzx5mA_TyQnxrCNpn3EpFGSFtEt1mA311teSyWV4QgUAmOaUIiW8xLk6KV9zqoYOKgjK1x_qS1tD1WlYKGOFP9pXBH1H4=w2154-h536-no?authuser=0" alt="NGS_processing" height="250px">

　このテキストの例題では、簡易的なリードのクオリティコントロールをおこなっています。

<small>※ 通常、リードのクオリティコントロールから多型検出までの次世代シーケンス解析は、下記のようなソフトウェアでおこないます。
- リードのクオリティコントロール: fastp、FaQCs、Ttrimmomatic など
- リードのアライメント: bwa、bowtie2、hisat2 など
- 多型検出: samtools、bcftools、GATK など</small>



### テキストファイル処理の基本

　FASTAやFASTQは、記述形式こそ異なりますが、実際には単なる**テキストファイル**です。テキストエディタ（Windowsのメモ帳、Macのテキストエディットなど）のアプリケーションで開いて、そのファイルの内容を確認できます。

　Pythonなどのプログラムは基本的に、テキストファイルを最初の行から順番に1行ずつ読みます。プログラムでテキストファイルを処理する方法も同じです。

```
繰り返し構文を使って、最初の行から順番に1行ずつ処理する
```

　FASTA形式やFASTQ形式テキストファイルは、決められたルールに従って、数行分をひとつの塩基配列データとして記述されています。ライブラリなしでそれらのテキストファイルを処理することは可能ではありますが、**Biopython**という生物学データ用のライブラリを使うことでより簡単に取り扱うことができます。

　FASTA形式やFASTQ形式テキストファイルの処理の基本は、Biopythonを使うと次のように変わります。

```
繰り返し構文を使って、最初のデータから順番に1データずつ処理する
```

<img src="https://lh3.googleusercontent.com/pw/AM-JKLVcCNH-qKRCqOYFVdz8XyUv_mjVXp_GhMUjbolhunYjJxBQAUtKQDvYXhEp1oN1agRocS08LqZkFPf6-ZG_vJ5SJojHa9C0WOUunTGVolF5iKZdA6_f8BUsuF70vvLgJPBZllRlDIH1_add7t2AO3bY=w1999-h864-no?authuser=0" alt="Text_processing" height="300px">


### サンプルデータのダウンロード


　実習前に、下記のコードセルを実行して、サンプルデータをダウンロードしましょう。

サンプルデータ: 
- 
- [NGS_sample_mini_1.fastq](https://raw.githubusercontent.com/CropEvol/lecture/master/data/NGS_sample_mini_1.fastq)　（例題用） … 10個のリードデータが書かれたFASTQ
- [mini_genome.fasta](https://raw.githubusercontent.com/CropEvol/lecture/master/data/mini_genome.fasta)　（実習用） … ミニゲノムデータ。9本の染色体の塩基配列が書かれたFASTA

　ColabサーバーにはBiopythonがインストールされていません。Biopythonのインストールも一緒におこなっておきましょう。

<small>※ 以下は、Pythonプログラムではありません。行頭に`!`を付けて、UNIXコマンド（パソコンやサーバーなどを操作・管理するコマンド）を実行しています</small>

In [ ]:
### ＊重要＊
### 最初にこのセルを実行してください。
### サンプルデータをダウンロードやライブラリのインストールをおこないます。

### サンプルデータのダウンロード
!wget -O "NGS_sample_mini_1.fastq" https://raw.githubusercontent.com/CropEvol/lecture/master/data/NGS_sample_mini_1.fastq
!wget -O "mini_genome.fasta"        https://raw.githubusercontent.com/CropEvol/lecture/master/data/mini_genome.fasta

### Biopythonのインストール
!pip install biopython

## 今回の実習内容

　このテキストでは、次のようなFASTQ処理プログラムを作成していきます。部品プログラムを徐々に追加し、ひとつのプログラムを完成させていきます。

```
FASTQ形式ファイルを読み込み、リードを順番に取り出し、平均クオリティが閾値以上のリードのみをファイル出力する
```

```python
# 例題のFASTQ処理プログラム概要
テキストファイルを開く

テキストデータをFASTQ形式として読み込む

リードを1個ずつ取り出す（繰り返し構文）:
  リードの長さを取得する
  リードのクオリティ情報を取得する
  リードの平均クオリティを計算する

  平均クオリティが閾値以上の場合（条件分岐構文）: 
    リードをファイルに書き出す

テキストファイルを閉じる
```

実習内容: 
1. ファイルを開く・閉じる
1. 1行ずつ読み込む
1. BiopythonでFASTQ形式データを扱う
  1. Biopythonライブラリを準備する
  1. FASTQ形式データとして読み込む
  1. リードデータを順番に取り出す
1. 各リードの情報にアクセスする
  1. リードの名前
  1. リードの塩基配列
  1. リードのクオリティ情報
1. 各リードの平均クオリティを調べる
  1. 塩基数を得る
  1. 各塩基のクオリティの合計値を得る
  1. 平均クオリティを計算する
1. 平均クオリティが一定以上のリードのみを表示する
1. 平均クオリティが一定以上のリードのみをファイルに書き出す

---


## 1. ファイルを開く・閉じる

　テキストファイルの中のデータを処理するためには、まずテキストファイルを開く必要があります。ここでは、テキストファイルを開いて、閉じる操作を学びます。

　テキストファイルを開くためには、`open`関数を使います。また、閉じるときには`close`関数を使います。

```python
ファイル変数 = open("<テキストファイルの名前>")
ファイル変数.close()
```

<small>※ `ファイル変数.close()`の`close()`は、ファイル変数が扱える「関数」です。より正確に言うと「メソッド」と呼ばれるものです。このメソッドを詳しく説明するには「オブジェクト志向」という少しややこしい話をする必要があります。時間の都合上、ゲノム情報解析入門では説明しません。関数とメソッドの違いを詳しく知りたい方は、Pythonに慣れてから自分で勉強してみてください。</small>

In [ ]:
### 例1
### ファイルを開く
f = open("NGS_sample_mini_1.fastq")  

### ファイル変数を確認
print(f)

### ファイルを閉じる
f.close()  

### 実習1

　実習では、FASTA形式ファイルを処理するプログラムを作っていきます。

```
FASTA形式ファイルを読み込み、塩基配列を順番に取り出し、長さが閾値以上の塩基配列のみをファイル出力する
```


　ここでは、open関数で実習用ファイル`mini_genome.fasta`を開くコードを追記してください。

<small>※ 次のコードセルの2行目に追記してください。</small>



In [ ]:
### ファイルを開く
F = 

### ファイル変数を確認
print(F)

### ファイルを閉じる
F.close()

#### 解答例

In [ ]:
### ファイルを開く
F = open("mini_genome.fasta")

### ファイル変数を確認
print(F)

### ファイルを閉じる
F.close()

## 2. 1行ずつ読み込む

　次は、前回習った繰り返し構文（`for`構文）を使って、テキストファイルの中身を1行ずつ表示してみましょう。

```python
ファイル変数 = open("<テキストファイルの名前>")
for 変数 in ファイル変数:
  print(変数)
ファイル変数.close()
```

In [ ]:
### 例2
### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### ループ処理: ファイルの中身を1行ずつ取り出されて、変数lineに入る
for line in f:
  print(line) # 表示

### ファイルを閉じる
f.close()

　行と行のあいだに、不要な空白行が挿入されてしまっています。これは、元々各行に含まれている**改行文字 (`\n`)**に加え、`print`関数により新たに改行文字が追加されてしまうためです。

<small>※ プログラミングをしていると、改行文字のような特殊文字に遭遇します。特殊文字は、表示されていませんが、ファイルには書き込まれています。</small>

<img src="https://lh3.googleusercontent.com/pw/AM-JKLW0ojbUAGTjeNlPeL0Rhe50NP50t8EeuzsDYHGFfsNmMZaZW40IYhn3LhYu15qeryBWPPN00nRM_hbsIfSmYpkDp96Ha4FoqvnU4KbSjSkMko75i9-cN2sIzLCeHNM6Fe8XX6KAtlSck_n16PKUOuoa=w987-h474-no?authuser=0" alt="newline_code" height="200px">

### 実習2

　for構文を使って、`mini_genome.fasta`の中身を1行ずつ表示してください。

<small>※ 次のコードセルの5行目あたりに追記してください。</small>

In [ ]:
### ファイルを開く
F = open("mini_genome.fasta")

### ループ処理


### ファイルを閉じる
F.close()

#### 解答例

In [ ]:
### ファイルを開く
F = open("mini_genome.fasta")  

### ループ処理
for line in F:
  print(line)

### ファイルを閉じる
F.close()  

## 3. BiopythonでFASTQ形式データを扱う

　Biopythonを使って、FASTQ形式データを扱えるようにしていきましょう。

### 3.1 Biopythonライブラリを準備する

　Pythonの追記機能（ライブラリ）を使うためには、そのライブラリを準備する必要があります。

　FASTA形式やFASTQ形式データを扱うには、Biopythonライブラリの`SeqIO`機能を使います。まず`SeqIO`を準備しましょう。

```python
# BiopythonライブラリのSeqIOを準備する
from Bio import SeqIO
```

In [ ]:
### 例3.1
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")


### ファイルを閉じる
f.close()

### 3.2 FASTQ形式データとして読み込む

　次に、開いたファイルをFASTQ形式として扱うためのプログラムを追加しましょう。

　ファイルの内容がFASTQ形式であることを指定するには、Biopythonの`SeqIO.parase`を使います。

```python
# BiopythonライブラリのSeqIOを準備する
from Bio import SeqIO

# 開いたファイルがFASTQ形式であることを認識させる
# 「f」はopen関数で開いたファイルの情報を格納した"ファイル変数"
recs = SeqIO.parse(f, "fastq")

print(recs)  # 確認
```

In [ ]:
### 例3.2
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### ループ処理: ファイルの中身を1行ずつ取り出されて、変数lineに入る
#for line in f:
#  print(line) # 表示

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
print(recs)  # 確認

### ファイルを閉じる
f.close()

### 3.3 リードデータを順番に取り出す

 `for`構文を使って、リードを順番に取り出してみましょう。

```python
# for構文を使ってリードを順番に取り出す
for rec in recs:
  print(rec)      # リードの中身を確認
```

In [ ]:
### 例3.3
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  print(rec)      # 確認

### ファイルを閉じる
f.close()

### 実習3

　Biopythonを使った次の3つのコードを追加して、`mini_genome.fasta`の中身を1データずつ表示してください。

1. Biopythonの`SeqIO`を準備する: `from Bio import SeqIO`
2. FASTA形式ファイルとして読み込む: `recs = SeqIO.parse(ファイル変数, "fasta")`
3. 繰り返し構文で1データずつ取り出し、表示する:  

```python
for rec in recs:
  print(rec)
```

<small>※ 次のコードセルの2行目、8行目、11行目あたりに追記してください。</small>


In [ ]:
### BiopythonのSeqIOを準備する


### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む


### 繰り返し処理: 1データずつ取り出す


### ファイルを閉じる
F.close()

#### 解答例

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  print(rec)      # 確認

### ファイルを閉じる
F.close()

## 4. 各リードの情報にアクセスする

　`rec`のなかには以下の情報が含まれています。
- 1データ（リード）の名前（ID）
- 1データ（リード）の塩基配列（Seq）
- 1データ（リード）中の各塩基のクオリティ（Phred quality）

　それぞれの情報にアクセスしてみましょう。



### 4.1 リードの名前

　リードの名前を得るには次のコードを使用します。

```python
# リードの名前を得る
rec.id
```


In [ ]:
### 例4.1
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  print(rec.id)    # リード名

### ファイルを閉じる
f.close()

### 4.2. リードの塩基配列

　リードの塩基配列を得るには次のコードを使用します。

```python
# リードの塩基配列を得る
rec.seq
```


In [ ]:
### 例4.2
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  print(rec.id)    # リード名
  print(rec.seq)  # 塩基配列本体

### ファイルを閉じる
f.close()

### 4.3. リード中の各塩基のクオリティ情報

　各塩基のクオリティ情報を得るには次のコードを使用します。

```python
# 各塩基のクオリティ情報を得る
rec.letter_annotations["phred_quality"]
```


In [ ]:
### 例4.3
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  print(rec.id)    # リード名
  print(rec.seq)  # 塩基配列本体
  print(rec.letter_annotations["phred_quality"])  # 各塩基のクオリティ情報

### ファイルを閉じる
f.close()

### 実習4

　各塩基配列データの塩基配列本体を表示してください。塩基配列本体の取り出し方はFASTAQ形式のときと同じです。

<small>※ 次のコードセルの13行目に追記してください。</small>

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  print(seq.id) # 塩基配列の名前
  print() # 塩基配列本体

### ファイルを閉じる
F.close()

#### 解答例

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  print(rec.id) # 塩基配列の名前
  print(rec.seq) # 塩基配列本体

### ファイルを閉じる
F.close()

## 5. 各リードの平均クオリティを調べる

　各リードの平均クオリティは次の式で得られます。
```
リードの平均クオリティ = 各塩基のクオリティの合計値 / リードの長さ（塩基数）
```

したがって、リードの長さ（塩基数）と各塩基のクオリティの合計値の情報が必要です。


### 5.1. 塩基数を得る

　リードの長さ（塩基数）を得るには、Pythonの基礎でも使った`len`関数を使用します。

```python
# len関数
len(文字列) # 文字列の文字数を調べる
len(リスト)   # リストの要素数を調べる
```

<small>※ `len`関数にリストデータ（`各塩基のクオリティ情報`）を与えて、リストデータの要素数を取得してもOKです。</small>

In [ ]:
### 例5.1
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  #print(rec.id)    # リード名
  #print(rec.seq)  # 塩基配列本体
  #print(rec.letter_annotations["phred_quality"])  # 各塩基のクオリティ情報

  qual = rec.letter_annotations["phred_quality"]  # 各塩基のクオリティ情報を変数に入れる
  seq_length = len(rec.seq)   # リードの長さ
  print('L=', seq_length)         # 確認

### ファイルを閉じる
f.close()

### 5.2. 各塩基のクオリティの合計値を得る

　各塩基のクオリティの合計値を得るには`sum`関数を使います。

```python
# sum関数
sum(リストデータ)
```

In [ ]:
### 例5.2
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  #print(rec.id)    # リード名
  #print(rec.seq)  # 塩基配列本体
  #print(rec.letter_annotations["phred_quality"])  # 各塩基のクオリティ情報

  qual = rec.letter_annotations["phred_quality"]  # 各塩基のクオリティ情報を変数に入れる
  seq_length = len(rec.seq)   # リードの長さ
  total_qual  = sum(qual)  # 各塩基クオリティの合計
  print('L=', seq_length, '; TotalQ=', total_qual)         # 確認

### ファイルを閉じる
f.close()

### 5.3. 平均クオリティを計算する

　リードの平均クオリティを計算しましょう。

```
リードの平均クオリティ = 各塩基のクオリティの合計値 / リードの長さ（塩基数）
```


In [ ]:
### 例5.3
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  #print(rec.id)    # リード名
  #print(rec.seq)  # 塩基配列本体
  #print(rec.letter_annotations["phred_quality"])  # 各塩基のクオリティ情報

  qual = rec.letter_annotations["phred_quality"]  # 各塩基のクオリティ情報を変数に入れる
  seq_length = len(rec.seq)   # リードの長さ
  total_qual  = sum(qual)  # 各塩基のクオリティの合計
  average_qual = total_qual / seq_length  # リードの平均クオリティ
  print('L=', seq_length, '; TotalQ=', total_qual, '; AverageQ=', average_qual)         # 確認

### ファイルを閉じる
f.close()

### 実習5

　各塩基配列の長さを調べてください。

<small>※ 次のコードセルの16行目に追記してください。</small>

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  print(rec.id) # 塩基配列の名前
  #print(rec.seq) # 塩基配列本体

  ### 塩基配列の長さを調べる
  seq_length = 
  print(seq_length)

### ファイルを閉じる
F.close()

#### 解答例

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  print(rec.id) # 塩基配列の名前
  #print(rec.seq) # 塩基配列本体

  ### 塩基配列の長さを調べる
  seq_length = len(rec.seq)
  print(seq_length)

### ファイルを閉じる
F.close()

## 6. 平均クオリティが一定以上のリードのみを表示する
　平均クオリティが30未満の場合には何も処理をせず、平均クオリティが30以上の場合にリードの名前を出力してみましょう。

　if構文（条件分岐構文）を使って、条件「リードの平均クオリティが30以上かどうか」をチェックし、その条件がTrueのときにリードの名前を出力するコードを書きます。

```python
# if構文: 条件がTrueのときに処理を実行する
if 条件:
  処理

# コード例
if average_qual >= 30:
  print(seq.id)
```

In [ ]:
### 例6
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  #print(rec.id)    # リード名
  #print(rec.seq)  # 塩基配列本体
  #print(rec.letter_annotations["phred_quality"])  # 各塩基のクオリティ情報

  qual = rec.letter_annotations["phred_quality"]  # 各塩基のクオリティ情報を変数に入れる
  seq_length = len(rec.seq)   # リードの長さ
  total_qual  = sum(qual)  # 各塩基クオリティの合計
  average_qual = total_qual / seq_length  # リードの平均クオリティ
  #print(seq_length, total_qual, average_qual)         # 確認

  ### 条件分岐処理: リードの平均クオリティが一定以上かどうかの処理
  if average_qual >= 30:
    print(rec.id)                  # 確認： 平均クオリティが一定以上のリード名

### ファイルを閉じる
f.close()

### 実習6

　「塩基数が300以上」の塩基配列の名前を表示してください。

<small>※ 次のコードセルの20行目あたりに追記してください。</small>

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  #print(rec.id) # 塩基配列の名前
  #print(rec.seq) # 塩基配列本体

  ### 塩基配列の長さを調べる
  seq_length = len(rec.seq)
  #print(seq_length)

  ### 塩基数が300以上であれば、塩基配列の名前を表示する


### ファイルを閉じる
F.close()

#### 解答例

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")  

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  #print(rec.id) # 塩基配列の名前
  #print(rec.seq) # 塩基配列本体

  ### 塩基配列の長さを調べる
  seq_length = len(rec.seq)
  #print(seq_length)

  ### 塩基数が300以上であれば、塩基配列の名前を表示する
  if seq_length >= 300:
    print(rec.id)

### ファイルを閉じる
F.close()

## 7. 平均クオリティが一定以上のリードのみをファイルに書き出す

　最後に、条件（平均クオリティが30以上）に合ったリードをファイルに書き出してみましょう。

手順: 
1. 新規の書き込み用ファイルを作成します。
1. Biopythonの`SeqIO.write()`を使って、FASTQ形式でファイルに出力します。
1. 作成した書き込み用ファイルを閉じます。

```python
# FASTQ形式でのファイル出力
# 書き込み用ファイルを作成（「mode=」は省略可）
ファイル変数 = open(ファイル名, mode="w") 

# 書き込み用ファイルにFASTQ形式データを出力
SeqIO.write(データ, handle=ファイル変数, format="fastq")

# 書き込み用ファイルを閉じる
ファイル変数.close()
```

In [ ]:
### 例7
from Bio import SeqIO

### ファイルを開く
f = open("NGS_sample_mini_1.fastq")
o = open("NGS_sample_mini_1_filtered.fastq", mode="w") # 書き込み可能ファイルを作成

### BiopythonでFASTQファイルを読み込む
recs = SeqIO.parse(f, "fastq")
#print(recs)  # 確認

### 繰り返し処理: FASTQファイルから1リードデータずつ取り出す
for rec in recs:
  #print(rec)      # 確認
  #print(rec.id)    # リード名
  #print(rec.seq)  # 塩基配列本体
  #print(rec.letter_annotations["phred_quality"])  # 各塩基のクオリティ情報

  qual = rec.letter_annotations["phred_quality"]  # 各塩基のクオリティ情報を変数に入れる
  seq_length = len(qual)   # 塩基配列の長さ
  total_qual  = sum(qual)  # 各塩基クオリティの合計
  average_qual = total_qual / seq_length  # リードの平均クオリティ
  #print(seq_length, total_qual, average_qual)         # 確認

  ### 条件分岐処理: リードの平均クオリティが一定以上かどうかの処理
  if average_qual >= 30:
    print(rec.id)                  # 確認： 平均クオリティが一定以上のリード名
    SeqIO.write(rec, handle=o, format="fastq")   # 平均クオリティが一定以上のリードをファイルに書き出す

### ファイルを閉じる
f.close()
o.close()

### 実習7

　次のコードセルにコードを追記して、「塩基数が300以上」の塩基配列をファイル`"mini_genome_len300.fasta"`に書き出してください。

```python
# FASTA形式での出力
SeqIO.write(データ, handle=ファイル変数, format="fasta")
```


<small>※ 次のコードセルの23行目あたりに適宜行を追加して、コードを追記してください。</small>


In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")
O = open("mini_genome_len300.fasta", mode="w") # 書き込み可能ファイルを作成

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  #print(rec.id) # 塩基配列の名前
  #print(rec.seq) # 塩基配列本体

  ### 塩基配列の長さを調べる
  seq_length = len(rec.seq)
  #print(seq_length)

  ### 塩基数が300以上であれば、塩基配列の名前を表示する
  if seq_length >= 300:
    print(rec.id)

### ファイルを閉じる
F.close()
O.close()

#### 解答例

In [ ]:
### BiopythonのSeqIOを準備する
from Bio import SeqIO

### ファイルを開く
F = open("mini_genome.fasta")
O = open("mini_genome_len300.fasta", mode="w") # 書き込み可能ファイルを作成

### FASTA形式ファイルとして読み込む
recs = SeqIO.parse(F, "fasta")

### 繰り返し処理: 1データずつ取り出す
for rec in recs:
  #print(rec.id) # 塩基配列の名前
  #print(rec.seq) # 塩基配列本体

  ### 塩基配列の長さを調べる
  seq_length = len(rec.seq)
  #print(seq_length)

  ### 塩基数が300以上であれば、塩基配列の名前を表示する
  if seq_length >= 300:
    print(rec.id)

    ### FASTA形式でファイルに出力する
    SeqIO.write(rec, handle=O, format="fasta")

### ファイルを閉じる
F.close()
O.close()

---
## まとめ

　Pythonの基礎で学んだことと、このテキストで新しく学んだ**Biopython**ライブラリを使って、塩基配列データファイル（**FASTQ**形式や**FASTA**形式ファイル）の処理を勉強しました。

　ここでおこなったFASTQ形式ファイルの処理は、次世代シーケンス解析のなかでは、**クオリティコントロール**に相当する処理です。クオリティコントロール後の品質の良いリードデータを使い、後続のゲノム解析をおこないます。

<small>※ 通常、クオリティコントロールは、既存のソフトウェア（fastpやFaQCs）でおこないます。</small>

　Biopythonは、FASTQやFASTA以外にも様々な形式の生物学データを扱うことができます。Biopython公式のチュートリアルにも詳しく記載されていますので、ぜひ勉強してみてください。

- Biopython公式チュートリアル: [Biopython Tutorial and Cookbook](http://biopython.org/DIST/docs/tutorial/Tutorial.html)


　PythonにはBiopython以外にも多数のライブラリがあります。次回、実際のゲノム解析でよく使用するライブラリ（**pandas**や**Matplotlib**）の使い方を勉強していきましょう。

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>